统计数据 长度 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("/home/extra1T/model_embeding/ZhipuAI/glm-4-9b-chat",trust_remote_code=True,)

model = AutoModelForCausalLM.from_pretrained(
    "/home/extra1T/model_embeding/ZhipuAI/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
import pdfplumber
with pdfplumber.open("/home/extra1T/kangh/app/slides.pdf") as pdf:
    first_page = pdf.pages[1]
    print(first_page.chars[1])

{'matrix': (28.000008, 0.0, 0.0, 28.000008, 483.3716, 428.88), 'fontname': 'AAAAAH+DengXian-Bold', 'adv': 1.0, 'upright': True, 'x0': 483.3716, 'y0': 422.383998144, 'x1': 511.371608, 'y1': 450.384006144, 'width': 28.00000799999998, 'height': 28.000008000000037, 'size': 28.000008000000037, 'mcid': None, 'tag': None, 'object_type': 'char', 'page_number': 2, 'ncs': 'ICCBased', 'text': '录', 'stroking_color': None, 'stroking_pattern': None, 'non_stroking_color': (1, 1, 1), 'non_stroking_pattern': None, 'top': 89.61599385599999, 'bottom': 117.61600185600003, 'doctop': 629.6159938559999}


In [1]:
import requests
import json
def get_weather(city_id):
    api_key = '451f1780d0d5ac1ac14ea48010d04f4d'
    url = f'https://dev.qweather.com/v7/weather/now?city={city_id}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if data['status'] == 200:
        weather_info = {
            'temperature': data['now']['tmp'],
            'humidity': data['now']['hum'],
            'wind_speed': data['now']['wind_spd'],
        }
        return weather_info
    else:
        return None

In [8]:
import pdfplumber
import pandas as pd

with pdfplumber.open("/home/extra1T/kangh/app/slides.pdf") as pdf:
    page = pdf.pages[10]   # 第一页的信息
    text = page.extract_text()
    print(text)
    table = page.extract_tables()
    for t in table:
        # 得到的table是嵌套list类型，转化成DataFrame更加方便查看和分析
        df = pd.DataFrame(t[1:], columns=t[0])
        print(df)


04 文 本 理 解
pdfplumber抽取文字、表格 首页或页眉获取招股书公司名 去除页眉页脚
数据预处理
保存为以公司名为标题的txt文件 转换表格数据为制表符间隔的文本 还原段落
公司名匹配 原公司名字符串匹配 bm25相似度召回，设置阈值 全文档字符串匹配
文本分割 langchain的RecursiveCharacterTextSplitter
提示词如下：
参数：chunk_size: 200 chunk_overlap: 100
------检索内容开始------
{extra_knowledge}
内容召回 bm25相似度召回，top_k：5,10,15,20,30
------检索内容结束------
用户问题：{user_question}。
完全根据检索内容结合问题回答用户问题，将问题和答案结合后
输出。注意不要输出“根据检索”。
模型输出 将检索信息和用户问题传入大模型得到输出
extra_knowledge: 检索到的信息 user_question: 用户问题


In [ ]:
import csv
import pandas as pd

# 读取输入的 CSV 文件
input_file = '/home/extra1T/jin/GoMate/test_question copy.csv'
df = pd.read_csv(input_file)
input_file1 = '/home/extra1T/jin/GoMate/test_question.csv'
df1 = pd.read_csv(input_file1)
# 创建输出 CSV 文件路径
output_file = '/home/extra1T/jin/GoMate/output_answers2.csv'

# 打开输出 CSV 文件，准备写入
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    # 写入表头
    writer.writerow(['question'])
    
    for row in range(265,len(df)):
        question = df['question'][row]
        question1 = df1['question'][row]
        messages = [{'role': 'user', 'content': question}]
        answers = gpt_35_api(messages)  # 返回列表类型
        answers = question1+answers
        
        combined_result = answers
        
        # 将组合后的字符串写入 CSV 文件
        writer.writerow([combined_result])
        # print(f"Processed: {combined_result}")

'你是一名高级智能助手，你需要根据当前提供的信息，执行当前任务。当前任务可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。若无需调用插件，直接执行任务，结果无需标志。\n1. {"name":"DocumentRetrieve","description":"招股书文档检索","parameters":[{"name":"company","description":"公司","required":false}]}\n\n2. {"name":"SqlGenerator","description":"sql语句生成","parameters":[{"name":"table_list","description":"所需表","required":true}]}\n\n3. {"name":"DatabaseQuery","description":"数据库查询","parameters":[{"name":"sql_sentence","description":"sql语句","required":true}]}\n\n用户问题：云南沃森生物技术股份有限公司负责产品研发的是什么部门？\n\n当前任务：查看云南沃森生物技术股份有限公司的公司招股书，寻找产品研发相关的信息。'

In [3]:
def analyze_txt_file(file_path):
    lengths = []
    # 项目
    # 读取文件内容并计算每行的长度
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_length = len(line.strip())  # 去除首尾空格
            if line_length > 0:  # 过滤掉空行
                lengths.append(line_length)
    
    # 计算平均长度、最小长度和最大长度
    if lengths:
        avg_length = sum(lengths) / len(lengths)
        min_length = min(lengths)
        max_length = max(lengths)
        max_line_number = lengths.index(max_length) + 1  # 行数从1开始
        
        print(f"第一行的长度为: {lengths[0]}")
        print(f"总共有 {len(lengths)} 行文本")
        print(f"平均长度为: {avg_length:.2f}")
        print(f"最短行的长度为: {min_length}")
        print(f"最长行的长度为: {max_length}，位于第 {max_line_number} 行")
    else:
        print("没有有效的文本行")

# 调用函数并传入你的文件路径
file_path = '/home/extra1T/jin/app/corpus.txt'
analyze_txt_file(file_path)


第一行的长度为: 181
总共有 50000 行文本
平均长度为: 900.67
最短行的长度为: 8
最长行的长度为: 46909，位于第 37969 行


In [ ]:
import json
import os

SOURCE_INDEX_DATA_PATH = "/home/extra1T/model_embeding/baike_qa_train.json"
OUTPUT_PATH = '/home/extra1T/kangh/app/kh/baike_qa.json'

# 加载并处理数据
source_index_data = []
with open(SOURCE_INDEX_DATA_PATH, encoding="utf8") as f:
    for line in f:
        ll = json.loads(line.strip())
        if len(ll["title"]) >= 2:
            source_index_data.append([ll["title"], ll])
        if len(ll["desc"]) >= 2:
            source_index_data.append([ll["desc"], ll])

# 只取前200000条数据
source_index_data = source_index_data[:100]

# 保存为json文件
with open(OUTPUT_PATH, 'w', encoding='utf8') as outfile:
    json.dump(source_index_data, outfile, ensure_ascii=False, indent=4)

print(f"数据已保存到 {OUTPUT_PATH}")


加载并且调用simcse-chinese-roberta-wwm-ext 向量模型

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from transformers import BertConfig, BertModel, BertTokenizer
# 使用本地模型路径
model_path = "/home/extra1T/model_embeding/simcse-chinese-roberta-wwm-ext"
query = ['你好呀','你是谁']
model = BertModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
q_id = tokenizer(query, max_length = 200, truncation=True, padding="max_length", return_tensors='pt')
out = model(**q_id)
print(q_id)
print(out)
out.last_hidden_state[:, 0].shape
# # 初始化 HuggingFaceEmbeddings，使用本地模型路径
# embedding = HuggingFaceEmbeddings(model_name=model_path)

# # 使用 Chroma 向量存储（示例，假设要创建 Chroma 数据库）
# vector_store = Chroma(embedding_function=embedding)

# # 示例：将文本转换为嵌入向量
# texts = ["今天天气很好", "我喜欢学习自然语言处理"]
# embeddings = [embedding.embed(text) for text in texts]

# # 打印嵌入
# print(embeddings)


/home/extra1T/kangh/miniconda3/agent/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


{'input_ids': tensor([[ 101,  872, 1962, 1435,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

torch.Size([2, 768])

视频里面的多路召回融合

In [ ]:
bge = json.load(open('submit bge sgement retrieval top10.json'))
bm25 = json.load(open('submit bm25 retrieval top10.json'))
fusion_result =[]
k=60
for q1,q2 in zip(bge, bm25):
    fusion_score ={}
    for idx,q in enumerate(q1['reference']):
            if q not in fusion_score:
                fusion_score[q]=1/(idx+ k)
            else:
                fusion_score[q]+=1/(idx + k)
    for idx,q in enumerate(q2['reference']):
            if q not in fusion_score:
                fusion_score[q]=1/(idx+ k)
            else:
                fusion_score[q]+=1/(idx + k)     
sorted_dict = sorted(fusion_score.items(), key=lambda item: item[1], reverse=True)
q1['reference'] = sorted_dict[0][0]
fusion_result.append(q1)

#bge 和 bm25的形状
# bge = [
#     {"reference": ["page_1", "page_2", "page_3"]},
#     {"reference": ["page_5", "page_1", "page_4"]}
# ]

# bm25 = [
#     {"reference": ["page_2", "page_1", "page_4"]},
#     {"reference": ["page_1", "page_6", "page_5"]}
# ]

BM25实现 方法 

In [ ]:
import jieba
from rank_bm25 import BM25Okapi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import torch
# 加载并处理文本
def load_and_process_txt(file_path):
    loader = TextLoader(file_path, encoding='utf8')
    documents = loader.load()
    
    # 分割文本，保持上下文一致性
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# 进行分词处理
def process_for_bm25(docs):
    # 对每个文档进行分词，得到一个二维数组
    tokenized_docs = [jieba.lcut(doc.page_content) for doc in docs]
    return tokenized_docs

# 文档路径
file_path = '/home/extra1T/jin/app/corpus copy.txt'

# 加载并分割文本
docs = load_and_process_txt(file_path)
# print("dsad:",len(docs))
# print("dsads:",docs[0].page_content)
# 对文档进行分词，准备给 BM25 使用
tokenized_docs = process_for_bm25(docs)

# print("dsad:",tokenized_docs[0])

# 初始化 BM25 模型
bm25 = BM25Okapi(tokenized_docs)

# 输入查询，查询的内容需要分词
query = "第8届南方新丝路模特大赛的招募活动是在哪里举行的？"
query_tokens = jieba.lcut(query)

# 使用 BM25 进行检索，得到每个文档的相关性分数
doc_scores = bm25.get_scores(query_tokens)
print(len(doc_scores))

# 设置要返回的 topk 个文档
topk = 3  # 例如返回前3个相似答案

# 获取分数最高的 topk 个文档索引
topk_indices = doc_scores.argsort()[-topk:][::-1]

# 获取分数最高的 topk 个文档内容
topk_matches = [docs[idx].page_content for idx in topk_indices]

# 输出 topk 个最相关的文档内容
print("最相关的文档内容：")
for i, match in enumerate(topk_matches, 1):
    print(f"Top {i}:\n{match}\n")


In [1]:
from modelscope.hub.snapshot_download import snapshot_download
model_dir = snapshot_download('ZhipuAI/glm-4-9b-chat', cache_dir='/home/extra1T/model_embeding/')

In [6]:
def max_score(s: str) -> int:
    # 计算字符串中 b 的总数量
    total_b = s.count('b')
    
    # 初始化变量
    left_a_count = 0
    right_b_count = total_b
    max_score = 0
    
    # 遍历字符串，尝试在每个位置进行分割
    for i in range(len(s) - 1):  # 最后一个字符不能作为分割点
        if s[i] == 'a':
            left_a_count += 1
        else:
            right_b_count -= 1
        
        # 计算当前得分
        current_score = left_a_count + right_b_count
        
        # 更新最大得分
        max_score = max(max_score, current_score)
    
    return max_score

# 示例测试
s = "abbbab"
print(max_score(s))  # 输出应为 5

5


In [ ]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-X2FikC3IChAiXXgE3bAO6mjhqDiXa6w2zsse0PGJUVnWGGEE",
    base_url="https://api.chatanywhere.tech"
    # base_url="https://api.chatanywhere.org/v1"
)

In [ ]:
import jieba
from rank_bm25 import BM25Okapi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import torch

# 载入并处理文本
def load_and_process_txt(file_path):
    loader = TextLoader(file_path, encoding='utf8')
    documents = loader.load()
    
    # 分割文本，保持上下文一致性
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs
# FAISS 向量召回
def create_faiss_index(docs):
    embeddings_model = HuggingFaceEmbeddings(model_name='/home/extra1T/model_embeding/ai-modelscope/gte-large-zh')
    vector_store = FAISS.from_documents(docs, embeddings_model)
    return vector_store
# 文档路径
file_path = '/home/extra1T/jin/app/corpus copy.txt'

# 加载并分割文本
docs = load_and_process_txt(file_path)

# 初始化 FAISS 向量检索
vector_store = create_faiss_index(docs)

query = "第8届南方新丝路模特大赛的招募活动是在哪里举行的？"

# 使用 FAISS 进行检索，并返回文档和对应的索引编号
result_with_score = vector_store.similarity_search_with_score(query, k=topk)
faiss_topk_matches = [(res.page_content, docs.index(res)) for res, score in result_with_score]  # 直接访问 page_content

print("FAISS 检索结果：")
for i, (content, idx) in enumerate(faiss_topk_matches, 1):
    print(f"Top {i}:\nContent: {content}\nIndex: {idx}\n")

分别使用Faiss 和 BM25 单路召回的结果

In [ ]:
import jieba
from rank_bm25 import BM25Okapi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import torch

# 载入并处理文本
def load_and_process_txt(file_path):
    loader = TextLoader(file_path, encoding='utf8')
    documents = loader.load()
    
    # 分割文本，保持上下文一致性
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# 分词处理
def process_for_bm25(docs):
    tokenized_docs = [jieba.lcut(doc.page_content) for doc in docs]
    return tokenized_docs

# FAISS 向量召回
def create_faiss_index(docs):
    embeddings_model = HuggingFaceEmbeddings(model_name='/home/extra1T/model_embeding/ai-modelscope/gte-large-zh')
    vector_store = FAISS.from_documents(docs, embeddings_model)
    return vector_store

# 文档路径
file_path = '/home/extra1T/jin/app/corpus copy.txt'

# 加载并分割文本
docs = load_and_process_txt(file_path)

# 处理文档并进行 BM25 分词
tokenized_docs = process_for_bm25(docs)

# 初始化 BM25 模型
bm25 = BM25Okapi(tokenized_docs)

# 初始化 FAISS 向量检索
vector_store = create_faiss_index(docs)

# 输入查询，查询内容需要分词
query = "第8届南方新丝路模特大赛的招募活动是在哪里举行的？"
query_tokens = jieba.lcut(query)

# 使用 BM25 进行检索
bm25_doc_scores = bm25.get_scores(query_tokens)

# 设置返回的 topk 个文档
topk = 3

# 获取 BM25 topk 文档和其索引编号
bm25_topk_indices = bm25_doc_scores.argsort()[-topk:][::-1]
bm25_topk_matches = [(docs[idx].page_content, idx) for idx in bm25_topk_indices]  # 返回内容和索引

# 使用 FAISS 进行检索，并返回文档和对应的索引编号
result_with_score = vector_store.similarity_search_with_score(query, k=topk)
faiss_topk_matches = [(res.page_content, docs.index(res)) for res, score in result_with_score]  # 直接访问 page_content

# 打印结果
print("BM25 检索结果：")
for i, (content, idx) in enumerate(bm25_topk_matches, 1):
    print(f"Top {i}:\nContent: {content}\nIndex: {idx}\n")

print("FAISS 检索结果：")
for i, (content, idx) in enumerate(faiss_topk_matches, 1):
    print(f"Top {i}:\nContent: {content}\nIndex: {idx}\n")

# 返回结果
bm25_topk_matches, faiss_topk_matches


使用RRF进行多路召回后的融合 将BM25和Faiss进行多路召回融合

In [ ]:
import jieba
from rank_bm25 import BM25Okapi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# 载入并处理文本
def load_and_process_txt(file_path):
    loader = TextLoader(file_path, encoding='utf8')
    documents = loader.load()
    
    # 分割文本，保持上下文一致性
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# 分词处理
def process_for_bm25(docs):
    tokenized_docs = [jieba.lcut(doc.page_content) for doc in docs]
    return tokenized_docs

# FAISS 向量召回
def create_faiss_index(docs):
    embeddings_model = HuggingFaceEmbeddings(model_name='/home/extra1T/model_embeding/ai-modelscope/gte-large-zh')
    vector_store = FAISS.from_documents(docs, embeddings_model)
    return vector_store

# RRF 融合算法实现
def rrf_fusion(bm25_results, faiss_results, k=60):
    fusion_scores = {}

    # BM25 结果，计算 RRF 分数
    for rank, (doc, idx) in enumerate(bm25_results):
        if idx not in fusion_scores:
            fusion_scores[idx] = 1 / (k + rank)
        else:
            fusion_scores[idx] += 1 / (k + rank)

    # FAISS 结果，计算 RRF 分数
    for rank, (doc, idx) in enumerate(faiss_results):
        if idx not in fusion_scores:
            fusion_scores[idx] = 1 / (k + rank)
        else:
            fusion_scores[idx] += 1 / (k + rank)

    # 根据融合分数对文档进行排序
    sorted_fusion_results = sorted(fusion_scores.items(), key=lambda item: item[1], reverse=True)
    return sorted_fusion_results

# 文档路径
file_path = '/home/extra1T/jin/app/corpus copy.txt'

# 加载并分割文本
docs = load_and_process_txt(file_path)

# 处理文档并进行 BM25 分词
tokenized_docs = process_for_bm25(docs)

# 初始化 BM25 模型
bm25 = BM25Okapi(tokenized_docs)

# 初始化 FAISS 向量检索
vector_store = create_faiss_index(docs)

# 输入查询，查询内容需要分词
query = "第8届南方新丝路模特大赛的招募活动是在哪里举行的？"
query_tokens = jieba.lcut(query)

# 使用 BM25 进行检索
bm25_doc_scores = bm25.get_scores(query_tokens)

# 设置返回的 top K 个文档
# topk = 60

# 获取 BM25 top K 文档和其索引编号
bm25_topk_indices = bm25_doc_scores.argsort()[::-1]
bm25_topk_matches = [(docs[idx].page_content, idx) for idx in bm25_topk_indices]

# 使用 FAISS 进行检索，并返回文档和对应的索引编号
result_with_score = vector_store.similarity_search_with_score(query, k=len(docs))
faiss_topk_matches = [(res.page_content, docs.index(res)) for res, score in result_with_score]  # 直接访问 page_content
# 不在 BM25 检索时限制 top K，获取全部文档的评分和索引

# bm25_topk_matches
# faiss_topk_matches
# 使用 RRF 进行分数融合
rrf_results = rrf_fusion(bm25_topk_matches, faiss_topk_matches, k=60)
topk = 3
# 输出最终融合后的 top K 结果
print("RRF 融合结果：")
for idx, (doc_idx, score) in enumerate(rrf_results[:topk], 1):
    print(f"Top {idx}:\nContent: {docs[doc_idx].page_content}\nRRF Score: {score}\n")



多路召回后使用bge-reranker-base进行重排序操作 

In [ ]:
# 使用 BGE 模型进行重排序
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# 使用 RRF 进行分数融合
# rrf_results = rrf_fusion(bm25_topk_matches, faiss_topk_matches, k=60)

# 获取融合后 top 3 的文档
top3_docs = [docs[idx].page_content for idx, _ in rrf_results[:3]]
def bge_rerank(query, docs):
    # 加载模型和分词器
    tokenizer = AutoTokenizer.from_pretrained("/home/extra1T/model_embeding/Xorbits/bge-reranker-base")
    model = AutoModelForSequenceClassification.from_pretrained("/home/extra1T/model_embeding/Xorbits/bge-reranker-base")
    
    # 预处理
    inputs = tokenizer([query] * len(docs), docs, padding=True, truncation=True, return_tensors="pt",max_length=512)
    
    # 使用模型进行推理
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取重排序得分（logits），并按得分从高到低排序
    scores = outputs.logits.squeeze().tolist()
    reranked_results = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    
    return reranked_results

# 使用 BGE 模型对 RRF 融合后的 top3 进行重排序
reranked_results = bge_rerank(query, top3_docs)

# 输出重排序后的 top 3 结果
print("重排序后的 Top 3 结果：")
for doc, score in reranked_results:
    print(f"文档: {doc}, 得分: {score}")


In [ ]:
topk_fused_docs = [doc[0] for doc in reranked_results[:3]]


视频中的重排序逻辑

In [2]:
import jieba, json, pdfplumber
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from rank_bm25 import BM25Okapi

questions = json.load(open("questions.json"))

pdf = pdfplumber.open("初赛训练数据集.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })

# 加载重排序模型
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/home/extra1T/model_embeding/Xorbits/bge-reranker-base')
rerank_model = AutoModelForSequenceClassification.from_pretrained('/home/extra1T/model_embeding/Xorbits/bge-reranker-base')
rerank_model.cuda()

pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

for query_idx in range(len(questions)):
		# 首先进行BM25检索
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idxs = doc_scores.argsort()[-3:]
		
		# top3进行重排序
    pairs = []
    for idx in max_score_page_idxs:
        pairs.append([questions[query_idx]["question"], pdf_content[idx]['content']])

    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        inputs = {key: inputs[key].cuda() for key in inputs.keys()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()

    max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

查看 FAISS 向量库内部计算相似度的索引方法

In [ ]:
#这是 FAISS 中的一种索引类型，IndexFlatL2 表示使用 欧氏距离（L2 距离） 进行相似度计算。
#在 FAISS 中，IndexFlatL2 会存储所有的向量，并使用欧氏距离来进行相似度计算。值越小证明相似度越高 
vector_store.index

从建立好的 Faiss向量库中 进行查询 

In [ ]:
query = "在昆明中院联合晋宁县法院进行的公判中，为什么李佳宜被判无期徒刑？"
result = vector_store.similarity_search(query ,k = 10)
result

官方的Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate
template = """你是一个用于问答任务的助手。
使用下面检索到的上下文片段来完整的回答用户的问题。
如果你不知道答案，只需说你不知道。
问题：{question}
上下文：{context}
答案：
"""
prompt = ChatPromptTemplate.from_template(template)
prompt

加载模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
import torch
from transformers import pipeline
# model_path = "/home/extra1T/model_embeding/qwen/Qwen2-7B"
model_path = "/home/extra1T/model_embeding/ZhipuAI/glm-4-9b"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto",offload_folder="offload")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 设置 GPU 编号，如果单机单卡指定一个，单机多卡指定多个 GPU 编号
MODEL_PATH = "/home/extra1T/model_embeding/ZhipuAI/glm-4-9b"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

query = "你好"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

In [ ]:
inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto"
).eval()

In [ ]:
gen_kwargs = {"max_length": 250, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [2]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

# 假设你已经有了经过融合和重排序后的 top 3 结果
# 这里使用 rrf_results 中的 top 3 作为上下文文档
topk_fused_docs = [doc[0] for doc in reranked_results[:3]]  # 获取融合后 top 3 的文档内容

# 将 top 3 的文档内容连接起来作为上下文
context = "\n\n".join(topk_fused_docs)

# 定义 RAG 的 Prompt 模板，包含问题和上下文
prompt_template = """你是一个用于问答任务的助手。
使用下面检索到的上下文片段来完整的回答用户的问题。
如果你不知道答案，只需说你不知道。
{context}

问题: {question}
回答:"""

# 定义 Prompt
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


# 构建链条
rag_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# 输入查询问题
query = "第8届南方新丝路模特大赛的海选时间是什么？"

# 调用 RAG 链，传递经过多路召回和重排序后的 top 3 文档作为上下文
raw_answer = rag_chain.run({"context": context, "question": query})

# 输出答案
print(raw_answer)


批量循环处理所有问题 脚本 

In [11]:
import jieba
from rank_bm25 import BM25Okapi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import csv
# 载入并处理文本
def load_and_process_txt(file_path):
    loader = TextLoader(file_path, encoding='utf8')
    documents = loader.load()
    
    # 分割文本，保持上下文一致性
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# 分词处理
def process_for_bm25(docs):
    tokenized_docs = [jieba.lcut(doc.page_content) for doc in docs]
    return tokenized_docs

# FAISS 向量召回
def create_faiss_index(docs):
    embeddings_model = HuggingFaceEmbeddings(model_name='/home/extra1T/model_embeding/ai-modelscope/gte-large-zh')
    vector_store = FAISS.from_documents(docs, embeddings_model)
    return vector_store

In [12]:
# RRF 融合算法实现
def rrf_fusion(bm25_results, faiss_results, k=60):
    fusion_scores = {}
    for rank, (doc, idx) in enumerate(bm25_results):
        if idx not in fusion_scores:
            fusion_scores[idx] = 1 / (k + rank)
        else:
            fusion_scores[idx] += 1 / (k + rank)
    for rank, (doc, idx) in enumerate(faiss_results):
        if idx not in fusion_scores:
            fusion_scores[idx] = 1 / (k + rank)
        else:
            fusion_scores[idx] += 1 / (k + rank)
    sorted_fusion_results = sorted(fusion_scores.items(), key=lambda item: item[1], reverse=True)
    return sorted_fusion_results

# BGE 重排序模型
def bge_rerank(query, docs):
    tokenizer = AutoTokenizer.from_pretrained("/home/extra1T/model_embeding/Xorbits/bge-reranker-base")
    model = AutoModelForSequenceClassification.from_pretrained("/home/extra1T/model_embeding/Xorbits/bge-reranker-base")
    inputs = tokenizer([query] * len(docs), docs, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = outputs.logits.squeeze().tolist()
    reranked_results = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return reranked_results

In [13]:
# 主逻辑：循环处理每个问题，生成上下文并批量处理
def process_questions(questions, docs, bm25, vector_store, topk=3):
    final_contexts = []  # 用于保存所有问题的上下文

    for query in questions:
        query_tokens = jieba.lcut(query)

        # BM25 召回
        bm25_doc_scores = bm25.get_scores(query_tokens)
        bm25_topk_indices = bm25_doc_scores.argsort()[::-1]
        bm25_topk_matches = [(docs[idx].page_content, idx) for idx in bm25_topk_indices]

        # FAISS 召回
        result_with_score = vector_store.similarity_search_with_score(query, k=len(docs))
        faiss_topk_matches = [(res.page_content, docs.index(res)) for res, score in result_with_score]

        # RRF 融合
        rrf_results = rrf_fusion(bm25_topk_matches, faiss_topk_matches, k=60)

        # 取融合后的 Top 3
        top3_docs = [docs[idx].page_content for idx, _ in rrf_results[:topk]]

        # 使用 BGE 模型进行重排序
        reranked_results = bge_rerank(query, top3_docs)

        # 获取重排序后的 Top 3 文档内容
        topk_fused_docs = [doc[0] for doc in reranked_results[:topk]]

        # 将 Top 3 的文档内容拼接成上下文
        context = "\n\n".join(topk_fused_docs)

        # 将每个问题和对应的上下文信息保存
        final_contexts.append({"query": query, "context": context})

    return final_contexts

In [14]:
# 加载文档和 BM25/FAISS 初始化
file_path = '/home/extra1T/jin/app/corpus copy.txt'
docs = load_and_process_txt(file_path)
tokenized_docs = process_for_bm25(docs)
bm25 = BM25Okapi(tokenized_docs)
vector_store = create_faiss_index(docs)

def load_questions(file_path):
    questions = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)  # 假设 CSV 文件有一列叫 "questions"
        for row in reader:
            questions.append(row['question'].strip())  # 读取并清理每一行的问题
    return questions

# 假设 questions.csv 文件路径为 '/path/to/questions.csv'
questions_file_path = '/home/extra1T/jin/app/test_question copy.csv'
questions = load_questions(questions_file_path)

# 处理所有问题，获取上下文
final_contexts = process_questions(questions, docs, bm25, vector_store)

In [ ]:
final_contexts

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
import torch
from transformers import pipeline
model_path = "/home/extra1T/model_embeding/qwen/Qwen2-7B"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto",offload_folder="offload")
tokenizer = AutoTokenizer.from_pretrained(model_path)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
import csv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain import PromptTemplate

# 假设你的 LLM 已经初始化好了
# llm = LLM()  # 这里的 LLM 实际上是你的语言模型实例

prompt_template = """你是一个用于问答任务的助手。
使用下面检索到的上下文片段来完整的回答用户的问题。
如果你不知道答案，只需说你不知道。
{context}

问题: {question}
回答:"""

# 定义 Prompt
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
# 构建链条
rag_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# 准备输出文件
output_file_path = '/home/extra1T/jin/appanswers.csv'
header = ['Question', 'Answer']

# 存储问题和答案到 CSV 文件
with open(output_file_path, 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)  # 写入表头

    for entry in final_contexts:
        query = entry['query']
        context = entry['context']
        
        # 将问题和上下文传递给大模型
        raw_answer = rag_chain.run({"context": context, "question": query})

        # 将问题和答案写入到 CSV 文件
        writer.writerow([query, raw_answer])

print(f"所有问题和答案已经写入到 {output_file_path}")


定义结构链条 结合LLM和prompt 以及结合到的知识库查询进行最终回答 

In [ ]:
import pandas as pd
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 假设 retriever, prompt 和 llm 已经定义好了
# 这里保留 rag_chain 逻辑
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 读取CSV文件
input_csv_path = '/home/extra1T/jin/app/test_question copy.csv'  # 输入问题的CSV文件路径
output_csv_path = '/home/extra1T/jin/app/output_answers.csv'  # 输出答案的CSV文件路径

# 读取CSV中的问题列
df = pd.read_csv(input_csv_path)

# 假设CSV中问题的列名是 'question'
questions = df['question'].tolist()

# 创建一个空列表存储问题的答案
answers = []

# 循环遍历每个问题并调用 rag_chain.invoke(query)
for question in questions:
    print(f"正在处理问题: {question}")
    try:
        answer = rag_chain.invoke(question)
        answers.append(answer)
    except Exception as e:
        print(f"处理问题 '{question}' 时出错: {e}")
        answers.append("处理失败")

# 将问题和答案存储到新的DataFrame
output_df = pd.DataFrame({
    'answer': answers
})

# 将DataFrame写入新的CSV文件
output_df.to_csv(output_csv_path, index=False)

print(f"所有问题处理完毕，结果已保存到 {output_csv_path}")


RMKV详细解释 

In [ ]:
########################################################################################################
# The RWKV Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=200)
import types, torch
from torch.nn import functional as F
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("/home/extra1T/model_embeding/20B_tokenizer.json")

args = types.SimpleNamespace()
args.MODEL_NAME = '/home/extra1T/model_embeding/RWKV-4-Pile-430M-20220808-8066'
args.n_layer = 24
args.n_embd = 1024

# context = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."
context = "On a sunset evening, a boy was waiting downstairs with a bouquet of flowers for the girl"
NUM_TRIALS = 3
LENGTH_PER_TRIAL = 100
TEMPERATURE = 1.0
TOP_P = 0.85

########################################################################################################

class RWKV_RNN(torch.jit.ScriptModule):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.eval() # set torch to inference mode
        
        w = torch.load(args.MODEL_NAME + '.pth', map_location='cpu')
        for k in w.keys():
            if      '.time_' in k: w[k] = w[k].squeeze()
            if '.time_decay' in k: w[k] = -torch.exp(w[k].float()) # the real time decay is like e^{-e^x}
            else: w[k] = w[k].float() # convert to f32 type
        
        self.w = types.SimpleNamespace() # set self.w from w
        self.w.blocks = {}
        for k in w.keys(): # example: "blocks.0.att.time_first" => self.w.blocks[0].att.time_first
            parts = k.split('.')
            last = parts.pop()
            here = self.w
            for p in parts:
                if p.isdigit():
                    p = int(p)
                    if p not in here: here[p] = types.SimpleNamespace()
                    here = here[p]
                else:
                    if not hasattr(here, p): setattr(here, p, types.SimpleNamespace())
                    here = getattr(here, p)
            setattr(here, last, w[k])

    def layer_norm(self, x, w):
        return F.layer_norm(x, (self.args.n_embd,), weight=w.weight, bias=w.bias)

    @torch.jit.script_method
    def channel_mixing(self, x, state, i:int, time_mix_k, time_mix_r, kw, vw, rw):
        xk = x * time_mix_k + state[5*i+0] * (1 - time_mix_k)
        xr = x * time_mix_r + state[5*i+0] * (1 - time_mix_r)
        state[5*i+0] = x
        r = torch.sigmoid(rw @ xr)
        k = torch.square(torch.relu(kw @ xk)) # square relu, primer paper
        return r * (vw @ k)

    @torch.jit.script_method
    def time_mixing(self, x, state, i:int, time_mix_k, time_mix_v, time_mix_r, time_first, time_decay, kw, vw, rw, ow):
        xk = x * time_mix_k + state[5*i+1] * (1 - time_mix_k)
        xv = x * time_mix_v + state[5*i+1] * (1 - time_mix_v)
        xr = x * time_mix_r + state[5*i+1] * (1 - time_mix_r)
        state[5*i+1] = x
        r = torch.sigmoid(rw @ xr)
        k = kw @ xk
        v = vw @ xv
        
        aa = state[5*i+2]
        bb = state[5*i+3]
        pp = state[5*i+4]
        ww = time_first + k
        qq = torch.maximum(pp, ww)
        e1 = torch.exp(pp - qq)
        e2 = torch.exp(ww - qq)
        a = e1 * aa + e2 * v
        b = e1 * bb + e2
        wkv = a / b
        ww = pp + time_decay
        qq = torch.maximum(ww, k)
        e1 = torch.exp(ww - qq)
        e2 = torch.exp(k - qq)
        state[5*i+2] = e1 * aa + e2 * v
        state[5*i+3] = e1 * bb + e2
        state[5*i+4] = qq
        return ow @ (r * wkv)

    def forward(self, token, state):
        with torch.no_grad():
            if state == None:
                state = torch.zeros(self.args.n_layer * 5, self.args.n_embd)
                for i in range(self.args.n_layer): state[5*i+4] = -1e30 # -infinity
            
            x = self.w.emb.weight[token]
            x = self.layer_norm(x, self.w.blocks[0].ln0)
            for i in range(self.args.n_layer):
                att = self.w.blocks[i].att
                x = x + self.time_mixing(self.layer_norm(x, self.w.blocks[i].ln1), state, i, 
                    att.time_mix_k, att.time_mix_v, att.time_mix_r, att.time_first, att.time_decay, 
                    att.key.weight, att.value.weight, att.receptance.weight, att.output.weight)
                ffn = self.w.blocks[i].ffn
                x = x + self.channel_mixing(self.layer_norm(x, self.w.blocks[i].ln2), state, i, 
                    ffn.time_mix_k, ffn.time_mix_r, 
                    ffn.key.weight, ffn.value.weight, ffn.receptance.weight)
            
            x = self.w.head.weight @ self.layer_norm(x, self.w.ln_out)
            return x.float(), state

##########################################################################################################

def sample_logits(out, temperature=1.0, top_p=0.8):
    probs = F.softmax(out, dim=-1).numpy()
    sorted_probs = np.sort(probs)[::-1]
    cumulative_probs = np.cumsum(sorted_probs)
    cutoff = float(sorted_probs[np.argmax(cumulative_probs > top_p)])
    probs[probs < cutoff] = 0
    if temperature != 1.0:
        probs = probs.pow(1.0 / temperature)
    probs = probs / np.sum(probs)
    out = np.random.choice(a=len(probs), p=probs)
    return out

########################################################################################################

print(f'\nUsing CPU. Loading {args.MODEL_NAME} ...')
model = RWKV_RNN(args)

print(f'\nPreprocessing context (slow version. see v2/rwkv/model.py for fast version)')
init_state = None
for token in tokenizer.encode(context).ids:
    init_out, init_state = model.forward(token, init_state)

for TRIAL in range(NUM_TRIALS):
    print(f'\n\n--[ Trial {TRIAL} ]-----------------', context, end="")
    all_tokens = []
    out_last = 0
    out, state = init_out.clone(), init_state.clone()
    for i in range(LENGTH_PER_TRIAL):
        token = sample_logits(out, TEMPERATURE, TOP_P)
        all_tokens += [token]
        tmp = tokenizer.decode(all_tokens[out_last:])
        if '\ufffd' not in tmp: # only print when we have a valid utf-8 string
            print(tmp, end="", flush=True)
            out_last = i + 1
        out, state = model.forward(token, state)       
print('\n')


In [ ]:
def max_sum_of_second_elements(arr):
    n = len(arr)
    if n < 3:
        return 0
    dp = [0] * n
    dp[2] = arr[1]  
    for i in range(3, n):
        dp[i] = dp[i-1]
        dp[i] = max(dp[i], dp[i-3] + arr[i-1])

    return dp[-1]

n = int(input("请输入数组的长度: "))
arr = list(map(int, input("请输入数组元素: ").split()))

print(max_sum_of_second_elements(arr))